## Survival Analysis: Based on eXtreme Gradient Boosting(XGB)

The tutorial give typical workflow of eXtreme Gradient Boosting-based survival analysis including data-preprocessing, model selection and traning&validation, uses **R package** `xgboost`.

Formally, it can be listed by:
1. Data Preprocessing
  - convert variables
  - load training and test set
2. Model Selection
  - cross validation
  - tune parameters
3. Traning&Validation
  - train gbm model
  - measure CI on testset
  - survival rates on time of interest

### Concept and Related

#### Concept of coxph
Under assumption of cox proportional hazard model, there is:
$$h(t|x) = h_0(t) \cdot e^{f(x)}$$
$f(x)$ is called **Hazard Ratio**.

Given the observation $\{(X_i,T_i,E_i)|i=1,\dots ,n \}$, the log partial likelihood function is:
$$
\mathcal L=-\sum_{i\ :\ E_i=1}\bigl(f(x_i)-log\sum_{j\in \mathcal R(i)}e^{f(x_j)}\bigr) \\
\mathcal R(i)=\{j|T_j\ge T_i\}
$$
If there is ties at time of death, the corresponding log partial likelihood function is:
$$
\mathcal L=-\sum_{i=1}^{k}\bigl(\sum_{j\in \mathcal q(i)}f(x_j)-d_i*log\sum_{j\in \mathcal R(i)}e^{f(x_j)}\bigr) \\
k = \big|\text{unique}(\{T_i|E_i=1\})\big| \\
\mathcal R(i)=\{j|T_j\ge T_i\} \\
\mathcal q(i)=\{j|T_j=T_i, E_i=1\} \\
d_i = \vert q(i) \vert
$$

### Step0 - Load library

In [1]:
library('survival')
library('xgboost')
library('gbm')
# set random state
set.seed(0)

Warning message:
"package 'xgboost' was built under R version 3.5.1"Warning message:
"package 'gbm' was built under R version 3.5.1"Loaded gbm 2.1.4


In [2]:
data(veteran, package = "randomForestSRC")
cat("Number of samples:", nrow(veteran), "\n")
cat("Columns of dataset:", colnames(veteran), "\n")
veteran[c(1:5), ]

Number of samples: 137 
Columns of dataset: trt celltype time status karno diagtime age prior 


trt,celltype,time,status,karno,diagtime,age,prior
1,1,72,1,60,7,69,0
1,1,411,1,70,5,64,10
1,1,228,1,60,3,38,0
1,1,126,1,60,9,63,10
1,1,118,1,70,11,65,10


### Step1 - Data Preprocessing

In [3]:
x_cols <- c('trt', 'celltype', 'karno', 'diagtime', 'age', 'prior')
y_cols <- c('T')
# Sample the data and create a training subset.
train <- sample(1:nrow(veteran), round(nrow(veteran) * 0.80))
data_train <- veteran[train, ]
data_test <- veteran[-train, ]

**Note**: format convertion

Data applied in survival analysis from package `xgboost` should be preproceed like that **new time label with sign** indicates censored or not(*negative values are considered right censored*).

In [4]:
# New column indicates time label with sign
data_train$T <- data_train$time
data_train$T[data_train$status==0] = -data_train$T[data_train$status==0]
data_test$T <- data_test$time
data_test$T[data_test$status==0] = -data_test$T[data_test$status==0]
# Convert to xgb.DMatrix
dtrain <- xgb.DMatrix(as.matrix(data_train[x_cols]), label=data_train[, y_cols])
dtest  <- xgb.DMatrix(as.matrix(data_test[x_cols]), label=data_test[, y_cols])

### Step2 - Model Selection

The hyperparameters list as follows:
- eta : learning rate
- nrounds : number of iterations
- max_depth : maximum depth of a tree
- min_child_weight: minimum sum of instance weight(hessian) needed in a child
- subsample: subsample ratio of the training instance
- colsample_bytree: subsample ratio of columns when constructing each tree
- gamma: minimum loss reduction required to make a further partition on a leaf node of the tree
- alpha: L1-Regularization of instance weight items
- labmda: L2-Regularization of instance weight items

Optional Reading: You can try to search the best hyperparameters' estimation by using python package `hyperopt`.

By the means of described, only `eta`, `max_depth` and `nrounds` were tuned, and repeated 4-fold cross validation on training set for 3 times is used, results of searching are:
- "eta": 0.035
- "max_depth": 1
- "nrounds": 90
- `CI` = 0.724952(average on repeated 4-fold cross validation for 3 times)

It's interesting that `CI` on test set is only about 0.65 by using hyperparameters searched by 4-cross validation. 

I think the main reason is that:
- Samples is not enough
- The randomized test set may be different from the training set distribution.

**Actually, the metrics on repeated k-fold cross validation is the best estimation of model's performance,** so using `CI` on k-fold cross validation to estimate performance and not spliting test set on small dataset is the best choice!

### Step3 - Model Training & Evaluation

We will pass arguments to object `xgboost` for training robust model after completing hyperparameters tuning in step2, and then validate our fitted model using test set.

Here, evaluation and more in this section includes:

- calculating CI metrics
- calculating survival function on specified time
- saving result as file

#### 3.0 - Model Training & Prediction

Note: returns of `predict` in `xgboost` is proportional hazard scale.

In [5]:
set.seed(2891)
param <- list(max_depth=3, eta = 0.06, silent = 1, objective = "survival:cox", eval_metric = "cox-nloglik")
model <- xgb.train(param, dtrain, nrounds=100)

In [6]:
pred.train <- log(predict(model, dtrain))
pred.test  <- log(predict(model, dtest))

### 3.1 - CI (concordance index)
We can get $\text{CI}$(concordance index) by function `rcorr.cens` from package `Hmisc`.

In [7]:
Hmisc::rcorr.cens(-pred.train, Surv(data_train$time, data_train$status))

C Index            Dxy           S.D.              n        missing 
  8.468341e-01   6.936682e-01   4.062630e-02   1.100000e+02   0.000000e+00 
    uncensored Relevant Pairs     Concordant      Uncertain 
  1.020000e+02   1.130800e+04   9.576000e+03   6.260000e+02

In [8]:
Hmisc::rcorr.cens(-pred.test, Surv(data_test$time, data_test$status))

C Index            Dxy           S.D.              n        missing 
     0.7335329      0.4670659      0.1384733     27.0000000      0.0000000 
    uncensored Relevant Pairs     Concordant      Uncertain 
    26.0000000    668.0000000    490.0000000     32.0000000

#### 3.2 - Survival function

Since `xgboost` don't have any funtion to obtain estimation of survival function, but `gbm` offers method `basehaz.gbm` to estimate the cumulative baseline hazard function $\int_0^{t}\lambda(z)dz$.  

Survival function can be estimated by:
$$
s(t|X)=exp{\{-\ e^{f(X)}\int_0^{t}\lambda(z)dz\}}
$$

**Note**: $f(X)$ is prediction of `xgboost`, which is **hazard proportion scale**.

So we can get survival function of individuals easily.

In [9]:
# Sepecify time of interest
time.interest <- sort(unique(data_train$time[data_train$status==1]))
# Estimate the cumulative baseline hazard function using training data
basehaz.cum <- basehaz.gbm(data_train$time, data_train$status, pred.train, t.eval = time.interest, cumulative = TRUE)

For individual $i$ in test set, estimation of survival function is:

In [10]:
surf.i <- exp(-exp(pred.test[1])*basehaz.cum)
print(surf.i)

 [1] 9.941249e-01 9.909250e-01 9.876193e-01 9.774264e-01 9.627323e-01
 [6] 9.544953e-01 9.500786e-01 9.456689e-01 9.367973e-01 9.322204e-01
[11] 9.275311e-01 9.130317e-01 9.075706e-01 8.964105e-01 8.901834e-01
[16] 8.774936e-01 8.642181e-01 8.567371e-01 8.491786e-01 8.332283e-01
[21] 8.171497e-01 8.090155e-01 8.006375e-01 7.917938e-01 7.828550e-01
[26] 7.739191e-01 7.649031e-01 7.556682e-01 7.458965e-01 7.153657e-01
[31] 6.932754e-01 6.818516e-01 6.592597e-01 6.477907e-01 6.362904e-01
[36] 6.239966e-01 6.115309e-01 5.986818e-01 5.726188e-01 5.582735e-01
[41] 5.434550e-01 5.286061e-01 5.131091e-01 4.973353e-01 4.638027e-01
[46] 4.455546e-01 4.271471e-01 4.088866e-01 3.905505e-01 3.556362e-01
[51] 3.340557e-01 3.129866e-01 2.925983e-01 2.720004e-01 2.510277e-01
[56] 2.309747e-01 2.106387e-01 1.901767e-01 1.700236e-01 1.507814e-01
[61] 1.316579e-01 1.128641e-01 9.514838e-02 7.903988e-02 6.474361e-02
[66] 5.087781e-02 3.881994e-02 2.837836e-02 1.973286e-02 1.275237e-02
[71] 8.031828e-03 4.

Estimation of survival rate of all at specified time is:

In [11]:
specif.time <- time.interest[10]
cat("Survival Rate of all at time", specif.time, "\n")
surv.rate <- exp(-exp(pred.test)*basehaz.cum[10])
print(surv.rate)

Survival Rate of all at time 15 
 [1] 0.9322204 0.9434882 0.7989306 0.8330214 0.9248614 0.9736593 0.7749758
 [8] 0.9421433 0.9079900 0.9702481 0.9657176 0.7250688 0.9587803 0.9814105
[15] 0.9908847 0.9359309 0.6546593 0.7648386 0.8964345 0.9563480 0.9380429
[22] 0.9523831 0.6165586 0.9108559 0.9507935 0.9206930 0.9610685


#### 3.3 - Saving as file

Here, we concate test data and prediction, survival rate, and then convert it to csv file.

In [12]:
res_test <- data_test
# predicted outcome for test set
res_test$pred <- pred.test
res_test$survival_rate <- surv.rate
# Save data
write.csv(res_test, file = "result_xgb.csv")